In [1]:
import os
import yaml

os.chdir('Z:/Covid19/Covid19_OMOP/new_pipeline/')
print(os.getcwd())

import pandas as pd
from omop_etl.load import Loader
# from .omop_etl.stage import STAGE

from pydma.utils import find

Z:\Covid19\Covid19_OMOP\new_pipeline


In [2]:
omop = Loader('config.yml')
STAGE = omop.stage

In [3]:
stage_t = omop.list_tables(in_schema=['stage'])
stage_t['Stage_RC'] = stage_t.Table.apply(lambda t: omop.row_count(t, schema='stage'))

In [4]:
stage_t_dict = stage_t[['Table','Stage_RC']].to_dict()
# stage_t_dict

In [5]:
stage_t_dict['Stage_RC'][7]

23734186

In [6]:
def find_key(input_dict, value):
    return next((k for k, v in input_dict.items() if v == value), None)

In [7]:
loaded = omop.config.load

result = []
for t in loaded.keys():
    if t not in ['provider','care_site','location']:
        if loaded[t]:
            for part in loaded[t]:
                stg_name = STAGE[t][part]
                idx = find_key(stage_t_dict['Table'], stg_name)
                count = stage_t_dict['Stage_RC'][idx]
                result.append([t, part, count])
        else:
            stg_name = STAGE[t]
            idx = find_key(stage_t_dict['Table'], stg_name)
            count = stage_t_dict['Stage_RC'][idx]
            result.append([t, None, count])

table_counts = pd.DataFrame(result, columns=['Table', 'Part', 'Stage_RC'])
# table_counts.head(50)

In [10]:
count_diff = table_counts.groupby('Table').sum().reset_index()
count_diff['Load_RC'] = count_diff.Table.apply(lambda t: omop.row_count(t))
count_diff['DeId_RC'] = count_diff.Table.apply(lambda t: omop.row_count(t, schema='hipaa'))
count_diff['Preload_RC'] = count_diff.Table.apply(lambda t: omop.row_count(t, schema='preload') if t not in ('death','person','visit_occurrence') else 0)
count_diff

,Table,Stage_RC,Load_RC,DeId_RC,Preload_RC
0,condition_occurrence,67002743,56801747,56801747,69174425
1,death,6301,6301,6301,0
2,drug_exposure,73551749,73408463,73408463,73444222
3,measurement,357397202,422814941,422814941,412434410
4,observation,71381467,87570857,87570857,71486920
5,person,204936,204935,204935,0
6,procedure_occurrence,26568160,12009481,12009481,26462556
7,visit_occurrence,23734186,23656621,23656621,0


In [9]:
# t_sbs = stage_t[find(['MEASUREMENT','DRUG','CONDITION','PROCEDURE','OBSERVATION'], stage_t.Table)].Table.tolist()